# Analyze morphology from segmented cells

by Felix Romer, inspired by Marie Münkel

last changed: 26.04.2025

Use the morphology_analysis environment

## Table of Contents

- [Package Import](#package-import)
- [Set Parameters](#set-parameters)
- [Other Variables and Preparation Steps](#other-variables-and-preparation-steps)
- [Analyze Morphology](#analyze-morphology)

## Package Import

In [1]:
# Data manipulation and analysis
import numpy as np
import pandas as pd 

# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns


# File handling
import os
import tifffile as tiff

# Image processing
from skimage.measure import regionprops_table
import feret
from skimage.measure import profile_line
from skimage.filters import gaussian
from skimage import io, color
from scipy.signal import find_peaks
from scipy.ndimage import binary_dilation

import Morphology_tools as MT


## Set Parameters

In [3]:
# Define directories
image_dirs      = ['/path/to/segmented/images_1/', 
                   '/path/to/segmented/images_2/']
actin_dirs      = ['/path/to/actin/images_1/',
                   '/path/to/actin/images_2/']
outdirs         = ['/path/to/save/results_1/',
                   '/path/to/save/results_2/']

# Set other parameters
file_formats    = ['.npy',
                   '.tif']
flow_directions = ['to_top', 
                   'to_top']

### Other Variables and Preparation Steps

In [ ]:
# List all files in the directory
all_files     = [os.listdir(image_dir) for image_dir in image_dirs]
all_actin_files = [os.listdir(actin_dir) for actin_dir in actin_dirs]

# Define parameters in a dictionary
analysis_params = {
    'name_region': '_region_stats.csv',
    'name_orientation': '_orientation.tif',
    'name_area': '_area.tif',
    'name_asp_ratio': '_asp_ratio.tif',
    'name_tortuosity': '_tortuosity.tif',
    'name_num_neighbors': '_num_neighbors.tif',
    'name_actin_fibers': '_actin_fibers.tif',
    'name_region_prop': '_region_properties.tif',
    'name_region_prop_numbers': '_region_properties_numbers.tif',
    'cmaps': [cm.inferno, cm.viridis, cm.viridis, cm.magma, cm.viridis, cm.cividis],
    'normalizations': [
        plt.Normalize(vmin=0, vmax=3000),
        plt.Normalize(vmin=0, vmax=90),
        plt.Normalize(vmin=0, vmax=5),
        plt.Normalize(vmin=3, vmax=5),
        plt.Normalize(vmin=0, vmax=10),
        plt.Normalize(vmin=0, vmax=20)
    ],
    'flow_direction_map': {
        "to_top": 0,
        "to_right": 1,
        "to_bottom": 2,
        "to_left": 3
    },
    'base_colors': {
        'Control': '#7c8083',
        'Infected': '#c53c37',
        'Uninfected': '#518fba'
    },
    # Save options
    'save_plots': True,
    'save_single_stats': True,
    'save_all_starts': True,
    'save_figure': True,
    # Crop image
    'crop_image': True,
    'crop_heigth': 700,
    # Area thresholds for filtering regions
    'min_area': 400,
    'max_area': 140000,
    # Calibration factor
    'fcal': 0.3478664,
    # Actin analysis
    'calculate_actin_fibers_number': True,
    # Display orientation
    'display_orientation': "to_right"
}


## Analyze Morphology

In [ ]:
# Loop over all directories (aka Experiment)
all_dfs = []
for i, params in enumerate(zip(image_dirs, outdirs, actin_dirs, all_files, all_actin_files, flow_directions, file_formats)):
    image_dir, outdir, actin_dir, files_total, actin_files, flow_direction, file_format = params
    print(f'Processing directory {i+1}/{len(image_dirs)}: {image_dir} \n')
    
    # Filter and sort the segmented image files
    if file_format == '.npy':
        files = sorted([str(f) for f in files_total if f.endswith('_seg.npy')])
    else:
        files = sorted([str(f) for f in files_total if f.endswith('.tif')])
    actin_files = sorted([str(f) for f in actin_files if f.endswith('.tif')])
    
    # Loop over each image file
    dfs = []
    for idx, (file, actin_file) in enumerate(zip(files, actin_files)):
        print(f'Processing file {idx+1}/{len(files)}: {file} \n')
        label_image, outlines, actin_image, name, filename, current_outdir = MT.process_and_initialize(file, actin_file, image_dir, actin_dir, outdir, flow_direction, analysis_params, file_format == '.npy')
        # make new dir
        current_outdir = outdir + name + '/'
        if not os.path.exists(current_outdir):
            os.makedirs(current_outdir)

        # Calculate region properties
        stats_filtered = MT.region_analysis(label_image, analysis_params)

        # Calculate actin fibers
        if analysis_params.get('calculate_actin_fibers_number') == True:
            fig, ax = plt.subplots(1,1)

            ax.imshow(label_image)
            for idx, region in stats_filtered.iterrows():
                region_label = region['label']
                #print(region_label)
                num_actin_fibers, _, _, _, intersections, min_feret_distance = MT.calculate_actin_fibers(region_label, actin_image, label_image, analysis_params.get('fcal'))
                stats_filtered.at[idx, 'num_actin_fibers']                   = num_actin_fibers
                stats_filtered.at[idx, 'actin_fibers_per_um']                = num_actin_fibers / min_feret_distance
                if len(intersections) >= 2:
                    start = intersections[0]
                    end   = intersections[1]
                    # Plot the line
                    ax.plot([start[1], end[1]], [start[0], end[0]], 'r-', lw=1)
            if analysis_params.get('save_plots'):
                # Save the plot
                plot_filename = current_outdir + name + '_linescan.tif'
                fig.savefig(plot_filename)

        stats_filtered['filename'] = name
        dfs.append(stats_filtered)
        
        #save to csv
        if analysis_params.get('save_single_stats'):
            stats_filtered.to_csv(filename, index=False)

        # Create visualization of region properties
        # TODO: Incude directly a if save_plots?
        prop_images = MT.create_propertie_image(stats_filtered, label_image, outlines, current_outdir, name, analysis_params)

        # Save the visualization as a figure
        if analysis_params.get('save_figure'):
            MT.save_as_figure(prop_images, stats_filtered, current_outdir, name, analysis_params)
    # Concatenate all DataFrames into a single DataFrame
    df_all = pd.concat(dfs, ignore_index=True)
    if analysis_params.get('save_all_starts'):
        df_all.to_csv(outdir + str(i) + 'all_regions.csv', index=False)
        print('Saved all regions to all_regions.csv')
    all_dfs.append(df_all)
# Concatenate all DataFrames into a single DataFrame
df_all = pd.concat(all_dfs, ignore_index=True)